In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import os
import cv2

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define path to dataset
path = '/content/drive/MyDrive/NeurIPS/all_eye_images'
dataset_path = os.listdir(path)
im_size = 224
images = []
labels = []

# Load and preprocess images
for i in dataset_path:
    data_path = path + '/' + str(i)
    filenames = [i for i in os.listdir(data_path)]
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

images = np.array(images)
images = images.astype('float32')
images /= 255.0

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = labels.reshape(-1, 1)

# One-hot encode labels
onehot_encoder = OneHotEncoder(sparse=False)
labels = onehot_encoder.fit_transform(labels)

# Shuffle and split data
images, labels = shuffle(images, labels, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.2, random_state=415)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.15, random_state=415)

# Data augmentation
train_generator = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_generator = ImageDataGenerator(rescale = 1.0/255.)
test_generator = ImageDataGenerator()

# Define Inception-ResNet model
base_model = InceptionResNetV2(input_shape=(im_size, im_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

inputs = layers.Input(shape=(im_size, im_size, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model summary
model.summary()

# Train model
history = model.fit(train_generator.flow(train_x, train_y, batch_size=32), epochs=13, callbacks=[early_stopping], validation_data=val_generator.flow(val_x, val_y, batch_size=32))

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(f'Test accuracy: {test_accuracy}')


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


219055592/219055592 [==============================] - 8s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d (  (None, 1536)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1573888   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                            